In [ ]:
import scvi
import scanpy as sc

In [ ]:
dir_path = "/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"
def load_data(dataset,batch):
    adata =sc.read_h5ad(dir_path+dataset+'.h5ad')
    sc.pp.filter_genes(adata, min_counts=3)
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata
    sc.pp.highly_variable_genes(
            adata,
            flavor="seurat",
            n_top_genes=2000,
            layer="counts",
            batch_key=batch,
            subset=True
    )

    scvi.data.setup_anndata(
        adata,
        layer="counts",
        batch_key=batch
    )
    return adata
    
batch_key_dic = {'Immune_Human' : 'batch',
                 'Immune_human_mouse' : 'batch',
                 'Lung' : 'batch',
                 'Mouse_brain' : 'batch',
                 'Pancreas' : 'Batch',
                 'Simulation1' : 'Batch',
                 'Simulation2' : 'Batch'}
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Immune_human_mouse' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Mouse_brain' : 'cell_type',
                 'Pancreas' : 'Group',
                 'Simulation1' : 'Group',
                 'Simulation2' : 'Group'}

In [ ]:
dataset = 'Immune_Human'
batch = batch_key_dic[dataset]
cell_type = cell_type_key_dic[dataset]
adata = load_data(dataset,batch)


In [ ]:
import time
start_time = time.time()

In [ ]:
# model.train(n_epochs=1)
model = scvi.model.SCVI(adata)
model.train()
adata.obsm["X_scVI"] = model.get_latent_representation()

In [ ]:
end_time = time.time()
print("total time taken: ", end_time-start_time)

In [ ]:
del adata.uns['_scvi']
adata.write('scVI-Immune_Human.h5ad')

In [ ]:
adata = sc.read_h5ad("scvi-Immune_Human.h5ad")

In [ ]:
#its a file in this folder
import scIB

In [ ]:
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all = scIB.metrics.metrics(
        adata,
        adata,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=True,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=True,
        embed='X_scVI',
        si_metric='euclidean',
        pcr_=True,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=True,  # backwards compatibility
        isolated_labels_f1_=True,
        isolated_labels_asw_=True,
        n_isolated=None,
        graph_conn_=True,
        kBET_=True,
        kBET_sub=0.5,
        lisi_graph_=True,
        lisi_raw=True,
        trajectory_=False,
        type_=None,
        verbose=True,
)

In [ ]:
results

In [ ]:
import numpy as np
np.savetxt(dataset+"_ilisi.csv", ilisi_all, delimiter=",")
np.savetxt(dataset+"_clisi.csv", clisi_all, delimiter=",")
np.savetxt(dataset+"_kbet_all.csv",np.concatenate([np.array(val).reshape(1,-1) for val in kbet_all],axis = 0), delimiter=',')

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi


latent_matrix = adata.obsm['X_scVI'] 
labels = np.array(adata.obs[cell_type] )
K = np.size(np.unique(labels))
kmeans = KMeans(n_clusters=K, random_state=0).fit(latent_matrix)
y_pred = kmeans.labels_

print('Computing NMI ...')
NMI = nmi(labels.flatten(), y_pred.flatten())
print('NMI = {}'.format(NMI))

In [ ]:
sc.pp.neighbors(adata, use_rep='X_scVI')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type, frameon=False)
sc.pl.umap(adata, color=batch, frameon=False)
sc.pl.umap(adata, color='cluster', frameon=False)

In [ ]:
adata.obs['temp'] =  y_pred
adata.obs['temp'] = adata.obs['temp'].astype(str)
sc.pl.umap(adata, color='temp', frameon=False)

In [ ]:
import scIB
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all = scIB.metrics.metrics(
        adata,
        adata,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=False,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=False,
        embed='X_scVI',
        si_metric='euclidean',
        pcr_=False,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=False,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=False,
        kBET_=False,
        kBET_sub=0.5,
        lisi_graph_=False,
        lisi_raw=False,
        trajectory_=True,
        type_=None,
        verbose=False,
)


In [ ]:
results

In [ ]:
import scIB
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all = scIB.metrics.metrics(
        adata,
        adata,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=True,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=False,
        embed='X_scVI',
        si_metric='euclidean',
        pcr_=False,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=False,  # backwards compatibility
        isolated_labels_f1_=False,
        isolated_labels_asw_=False,
        n_isolated=None,
        graph_conn_=False,
        kBET_=False,
        kBET_sub=0.5,
        lisi_graph_=False,
        lisi_raw=False,
        trajectory_=False,
        type_=None,
        verbose=False,
)
results

In [ ]:
sc.settings.figdir = '/home/krushna/Documents/Data_integration/Figures/Immune_Human/'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
sc.pp.neighbors(adata, use_rep='X_scVI')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=cell_type, frameon=False, save='_scvi_'+dataset+"_cell_type.png")
sc.pl.umap(adata, color=batch, frameon=False, save='_scvi_'+dataset+"_batch.png")
sc.pl.umap(adata, color='cluster', frameon=False, save='_scvi_'+dataset+"_optimal_resolution.png")

In [ ]:
import pandas as pd
import numpy as np
cluster_cell_type = pd.crosstab(np.array(adata.obs['cluster']),np.array(adata.obs['final_annotation']))
cluster_cell_type.index.name = 'cluster'
cluster_cell_type

In [ ]:
from scipy.stats import entropy

for ind,row in cluster_cell_type.iterrows():
    print(ind,entropy(row))


In [ ]:
li = []
for column in cluster_cell_type:
    li.append([column,entropy(cluster_cell_type[column])])
pd.DataFrame(li)